# Core

> Main functionality

In [ ]:
#| default_exp core

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#|export
import json, logging, re, requests

from bs4 import BeautifulSoup
from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util.retry import Retry

In [ ]:
#| export
class GithubDependentsInfo:
    "Main runner class"
    def __init__(
        self, 
        repo,
        sort_key = "name",
        min_stars = 0
    ):
        self.repo = repo
        self.url_init = f"https://github.com/{self.repo}/network/dependents"
        self.url_starts_with = f"/{self.repo}/network/dependents" + "?package_id="
        self.sort_key = sort_key
        self.min_stars = min_stars
        self.total_sum = 0
        self.total_public_sum = 0
        self.total_private_sum = 0
        self.total_stars_sum = 0
        self.packages = []
        self.all_public_dependent_repos = []
        self.result = {}

    def collect(self):
        # List packages
        self.compute_packages()

        # for each package, get count by parsing GitHub HTML
        for package in self.packages:
            nextExists = True
            result = []

            # Build start page url
            if package["id"] is not None:
                url = self.url_init + "?package_id=" + package["id"]
            else:
                url = self.url_init + ""
            package["url"] = url
            package["public_dependent_stars"] = 0
            page_number = 1

            # Get total number of dependents from UI
            r = self.requests_retry_session().get(url)
            soup = BeautifulSoup(r.content, "html.parser")
            svg_item = soup.find("svg", {"class": "octicon-code-square"})
            if svg_item is not None:
                a_around_svg = svg_item.parent
                total_dependents = self.get_int(a_around_svg.text.replace("Repositories", "").strip())
            else:
                total_dependents = 0
            

            # Parse all dependent packages pages
            while nextExists:
                r = self.requests_retry_session().get(url)
                soup = BeautifulSoup(r.content, "html.parser")
                total_public_stars = 0

                # Browse page dependents
                for t in soup.findAll("div", {"class": "Box-row"}):
                    result_item = {
                        "name": "{}/{}".format(
                            t.find("a", {"data-repository-hovercards-enabled": ""}).text,
                            t.find("a", {"data-hovercard-type": "repository"}).text,
                        ),
                        "stars": self.get_int(
                            t.find("svg", {"class": "octicon-star"}).parent.text.strip().replace(",", "")
                        ),
                    }
                    # Skip result if less than minimum stars
                    if self.min_stars is not None and result_item["stars"] < self.min_stars:
                        continue
                    result += [result_item]
                    total_public_stars += result_item["stars"]

                # Check next page
                nextExists = False
                paginate_container = soup.find("div", {"class": "paginate-container"})
                if paginate_container is not None:
                    for u in paginate_container.findAll("a"):
                        if u.text == "Next":
                            nextExists = True
                            url = u["href"]
                            page_number = page_number + 1

            # Manage results for package
            if self.sort_key == "stars":
                result = sorted(result, key=lambda d: d[self.sort_key], reverse=True)
            else:
                result = sorted(result, key=lambda d: d[self.sort_key])

            # Build package stats
            total_public_dependents = len(result)
            package["public_dependents"] = result
            package["public_dependents_number"] = total_public_dependents
            package["public_dependent_stars"] = total_public_stars
            package["private_dependents_number"] = total_dependents - total_public_dependents
            package["total_dependents_number"] = total_dependents if total_dependents > 0 else total_public_dependents

            # Build total stats
            self.all_public_dependent_repos += result
            self.total_sum += package["total_dependents_number"]
            self.total_public_sum += package["public_dependents_number"]
            self.total_private_sum += package["private_dependents_number"]
            self.total_stars_sum += package["public_dependent_stars"]


        # make all_dependent_repos unique
        self.all_public_dependent_repos = list({v["name"]: v for v in self.all_public_dependent_repos}.values())

        # Sort packages and dependent repos
        if self.sort_key == "stars":
            self.packages = sorted(self.packages, key=lambda d: d["public_dependent_stars"], reverse=True)
            self.all_public_dependent_repos = sorted(
                self.all_public_dependent_repos, key=lambda d: d["stars"], reverse=True
            )
        else:
            self.packages = sorted(self.packages, key=lambda d: d["name"])
            self.all_public_dependent_repos = sorted(self.all_public_dependent_repos, key=lambda d: d["name"])

        # Build final result
        return self.build_result()

    # Get first url to see if there are multiple packages
    def compute_packages(self):
        r = self.requests_retry_session().get(self.url_init)
        soup = BeautifulSoup(r.content, "html.parser")
        for a in soup.find_all("a", href=True):
            if a["href"].startswith(self.url_starts_with):
                package_id = a["href"].rsplit("=", 1)[1]
                package_name = a.find("span").text.strip()
                if "{{" in package_name:
                    continue
                self.packages += [{"id": package_id, "name": package_name}]
        if len(self.packages) == 0:
            self.packages = [{"id": None, "name": self.repo}]

    # Build result
    def build_result(self):
        self.result = {
            "all_public_dependent_repos": self.all_public_dependent_repos,
            "total_dependents_number": self.total_sum,
            "public_dependents_number": self.total_public_sum,
            "private_dependents_number": self.total_private_sum,
            "public_dependents_stars": self.total_stars_sum,
        }
        self.result["packages"] = (self.packages,)
        return self.result

    def requests_retry_session(
        self,
        retries=3,
        backoff_factor=0.5,
        status_forcelist=(500, 502, 504),
        session=None,
    ):
        session = session or requests.Session()
        retry = Retry(
            total=retries,
            read=retries,
            connect=retries,
            backoff_factor=backoff_factor,
            status_forcelist=status_forcelist,
        )
        adapter = HTTPAdapter(max_retries=retry)
        session.mount("http://", adapter)
        session.mount("https://", adapter)
        return session

    # Get integer from string
    def get_int(self, number_as_string: str):
        number_as_string = number_as_string.replace(",", "").replace(" ", "")
        return int(number_as_string)

In [ ]:
#| export
def get_deps(package_name:str):
    "Package is in form of user/repo"
    session = requests.Session()
    retry = Retry(
        total=3,
        read=3,
        connect=3,
        backoff_factor=0.5,
        status_forcelist=(500, 502, 504),
    )
    adapter = HTTPAdapter(max_retries=retry)
    session.mount("http://", adapter)
    session.mount("https://", adapter)
    nextExists = True
    total_packages = 0
    packages = []
    prefix = "https://github.com/"
    suffix = f"{package_name}/network/dependents?dependent_type=PACKAGE"
    url = prefix+suffix
    while nextExists:
        r = session.get(url)
        soup = BeautifulSoup(r.content, "html.parser")
        svg_item = soup.find("a", {"href": suffix})
        if svg_item is not None:
            total_packages = re.sub('[^0-9,]', "", svg_item.text)
        for a in soup.find_all("a", href=True):
            if a.get("data-hovercard-type", None) == "repository":
                repo = a.get("href")
                package = a.text
                packages += [{"id": repo, "name": package}]
        # Check next page
        nextExists = False
        paginate_container = soup.find("div", {"class": "paginate-container"})
        if paginate_container is not None:
            for u in paginate_container.findAll("a"):
                if u.text == "Next":
                    nextExists = True
                    url = u["href"]
    return packages

In [ ]:
#| export
def clean_data(results):
    for i,repo in enumerate(results["all_public_dependent_repos"]):
        name, stars = repo.values()
        if any(d['id'] == f"/{name}" for d in results["packages"]):
            results["all_public_dependent_repos"][i]["is_package"] = True
            for p in results["packages"]:
                if p["id"] == f"/{name}":
                    results["all_public_dependent_repos"][i]["package"] = p["name"]
        else:
            results["all_public_dependent_repos"][i]["is_package"] = False
            results["all_public_dependent_repos"][i]["package"] = None
    for p in results['all_public_dependent_repos']:
        p["author"], p["repo"] = p["name"].split("/", 1)
    return results

In [ ]:
#| export
def get_data(package_name):
    packages = get_deps("huggingface/accelerate")
    deps = GithubDependentsInfo("huggingface/accelerate")
    results = deps.collect()
    results["packages"] = packages
    results = clean_data(results)
    return results